In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler=StandardScaler()

In [5]:
scaler.fit(X_train)

StandardScaler()

In [6]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
from sklearn.neural_network import MLPRegressor

In [44]:
mlp_model = MLPRegressor(hidden_layer_sizes = (30,20,10,5)).fit(X_train_scaled, y_train)

In [45]:
mlp_model.n_layers_

6

In [46]:
mlp_model.hidden_layer_sizes

(30, 20, 10, 5)

In [47]:
y_pred = mlp_model.predict(X_test_scaled)

In [48]:
np.sqrt(mean_squared_error(y_test,y_pred))

360.41998510338476

In [49]:
mlp_params = {'alpha': [0.1, 0.01,0.02,0.005],
             'hidden_layer_sizes': [(20,20),(100,50,150),(300,200,150),(30,20,10,5)],
             'activation': ['relu','logistic']}

In [50]:
mlp_cv_model=GridSearchCV(mlp_model,mlp_params,cv=10)

In [51]:
mlp_cv_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(30, 20, 10, 5)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150),
                                                (30, 20, 10, 5)]})

In [52]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50, 150)}

In [54]:
mlp_tuned=MLPRegressor(activation= 'relu', alpha= 0.01, hidden_layer_sizes= (100, 50, 150))

In [55]:
mlp_tuned.fit(X_train_scaled, y_train)

MLPRegressor(alpha=0.01, hidden_layer_sizes=(100, 50, 150))

In [56]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [57]:
np.sqrt(mean_squared_error(y_test, y_pred))

356.2144335573618